In [ ]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm
import time
import json
from datetime import datetime, timedelta
# pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 100)

In [ ]:
data = pd.read_csv('../5_processing_extracted_data/plotdata.csv') # date,row_index,hardware_name,TH/J,max_efficiency,year

# Convert the 'date' column to a datetime format
data['date'] = pd.to_datetime(data['date'])

# keep only data from 2011 onwards
data = data[data['date'] >= '2011-01-02']

# Create a 'month' column
data['month'] = (data['date']).dt.to_period('M')

# Group by 'month' and calculate the mean of 'TH/J'
quarterly_data = data.groupby('month')['TH/J'].mean().reset_index()

# Convert 'month' back to datetime (first day of the month)
quarterly_data['month'] = quarterly_data['month'].dt.to_timestamp()

quarterly_data.to_csv('monthly_efficiency.csv', index=False)

In [ ]:
# Timestamp Graph,Intervals (Row),Network hashrate (TH/s),Bitcoins per block (BTC)
# 02/01/2009,1,4.97E-08,50
BitcoinData = pd.read_csv('BitcoinData.csv') 

df2 = pd.DataFrame(columns=['date', 'hashrate','coins_per_block'])

df2['date'] = pd.to_datetime(BitcoinData['Timestamp Graph'], format='%d/%m/%Y')
df2['hashrate'] = BitcoinData['Network hashrate (TH/s)']*1e12
df2['coins_per_block'] = BitcoinData['Bitcoins per block (BTC)']

df2 = df2.dropna()
df2 

In [ ]:
hashrate = json.load(open('hashrate.json'))

# x	y
# 1231545600000	1.065220e-07
BitcoinData2 = pd.DataFrame(hashrate["hash-rate"])
BitcoinData2["date"] = pd.to_datetime(BitcoinData2["x"], unit='ms')
BitcoinData2["hashrate"] = BitcoinData2["y"]*1e12
BitcoinData2 = BitcoinData2.drop(columns=["x", "y"])

In [ ]:
BitcoinData2

In [ ]:
temp = BitcoinData2[BitcoinData2["date"] > df2["date"].max()]
temp

In [ ]:


# df2 is not defined for values above 2022
df2 = pd.concat([df2, temp])

# if block is nan, it should be 6.25
df2['coins_per_block'] = df2['coins_per_block'].fillna(6.25)

In [ ]:
df2

In [ ]:
# for each month in df2, get the monthly average and store it with the date being the first day of the month

# Create a 'month' column
from datetime import timedelta
df2['month'] = (df2['date']).dt.to_period('M')

# Group by 'month' and calculate the mean
df2 = df2.groupby('month')[["hashrate","coins_per_block"]].mean().reset_index()

# Convert 'month' back to datetime (first day of the month)
df2['date'] = df2['month'].dt.to_timestamp()
df2 = df2.drop(columns=["month"])
df2

In [ ]:
# df2.to_csv('BitcoinData2.csv', index=False)

In [ ]:
# Start,End,Open,High,Low,Close,Volume,Market Cap
# 2024-03-08,2024-03-09,66871.2,69451.15,66398.03,68202.87,71117263448.78049,1329444482209.993
price = pd.read_csv('../../pricehistory/price_full.csv') 



# Date,ln(P_max),Data monthly average,Model ln(P_eff)
# 2011-04-01,-12.485786548332777,1.6612383008356546e-06,1.5657608695648479e-06
efficiency = pd.read_csv('monthly_efficiency.csv') 

df1 = pd.DataFrame(columns=['date', 'price'])
df3 = pd.DataFrame(columns=['date', 'efficiency'])


df1['date'] = pd.to_datetime(price['Start'])
df1['price'] = price['Close']

df3['date'] = pd.to_datetime(efficiency['month'], format='%Y-%m-%d')
df3['efficiency'] = efficiency['TH/J']*1e12

joined = pd.merge(df1, df2, on='date', how='inner')
joined = pd.merge(joined, df3, on='date', how='inner')
joined = joined.dropna()

In [ ]:
pd.set_option('display.max_rows', None)
joined.sample(10)

In [ ]:
# # joined["reward"] = joined["block"]*joined["price"]

# data = []

# for i in range(len(joined)):
#     date = joined['date'].iloc[i]
#     hashrate = joined['hashrate'].iloc[i]
#     efficiency = joined['efficiency'].iloc[i]
#     # reward = joined['reward'].iloc[i]
#     coins_per_block = joined['coins_per_block'].iloc[i]
#     price_per_coin = joined['price'].iloc[i]

#     aux = 2.0
#     electricity_cost = 0.05/(1000*3600)
#     constant = 6
#     watts = hashrate/efficiency
#     top = watts*aux*electricity_cost
#     coins_per_second = coins_per_block/600
#     bottom = constant*coins_per_second
#     cost_per_coin = top/bottom



#     joules_per_coin = watts/coins_per_block
#     joules_per_dollar_earned = joules_per_coin/price_per_coin

#     data.append([date, cost_per_coin, watts, coins_per_block, joules_per_coin,joules_per_dollar_earned])
    
#     # print(f"date: {date}")
#     # # print(f"hashrate: {hashrate}")
#     # # print(f"efficiency: {efficiency}")
#     # print(f"watts: {watts}")
#     # print(f"top: {top}")
#     # # print(f"reward: {reward}")
#     # print(f"bottom: {bottom}")
#     # print(f"cost: {cost_per_coin}")
#     # print("\n\n")

# df = pd.DataFrame(data, columns=['date', 'cost_per_coin', 'watts', 'coins_per_block', 'joules_per_coin','joules_per_dollar_earned'])

In [ ]:
# joined = joined.drop("price", axis=1)
df = joined

In [ ]:
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# from datetime import timedelta
# df['avg'] = (df['date'] - timedelta(days=15)).dt.to_period('Q')
# df.sample(10)

In [ ]:
# avg_data = df.groupby('avg')['cost'].mean().reset_index()
# avg_data

In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.date

In [ ]:
df.sample(1)

In [ ]:
max_efficiency_df = pd.read_csv('../../hardwarelist/Bitcoin max updated2.csv', usecols=['date', 'max (TH/J)'])
max_efficiency_df['date'] = pd.to_datetime(max_efficiency_df['date']).dt.date
# Create a 'month' column
max_efficiency_df['month'] = pd.to_datetime(max_efficiency_df['date']).dt.to_period('M')
# Group by 'month' and calculate the mean of 'TH/J'
max_efficiency_df = max_efficiency_df.groupby('month')['max (TH/J)'].mean().reset_index()
# Convert 'month' back to datetime (first day of the month)
max_efficiency_df['date'] = max_efficiency_df['month'].dt.to_timestamp()
max_efficiency_df = max_efficiency_df.rename(columns={'max (TH/J)': 'max_efficiency'})
max_efficiency_df.drop(columns=["month"], inplace=True)
max_efficiency_df.tail(5)

In [ ]:
general_df = pd.read_csv('../../bitcoinforum_general/5_processing_extracted_data/general.csv')
# keep only dates after 2011
general_df['date'] = pd.to_datetime(general_df['date']).dt.date
general_df = general_df[general_df['date'] >= datetime(2011, 1, 1).date()]
general_df.tail(5)

In [ ]:
# Merge with max efficiency
df['date'] = df['date'].astype(str)
max_efficiency_df['date'] = max_efficiency_df['date'].astype(str)
df = pd.merge(df, max_efficiency_df, on='date', how='left')

# Merge with general
# df['date'] = pd.to_datetime(df['date']).dt.date
# general_df['date'] = pd.to_datetime(general_df['date']).dt.date
df['date'] = df['date'].astype(str)
general_df['date'] = general_df['date'].astype(str)
df = pd.merge(df, general_df, on='date', how='left').fillna(method='ffill')

In [ ]:
sentiment_df = pd.read_csv('../../bitcoinforum_general/5_processing_extracted_data/normalized_values_sentiment.csv')
sentiment_df['date'] = pd.to_datetime(sentiment_df['month']).dt.date
sentiment_df.drop(columns=["month","strongly negative","strongly positive"], inplace=True)
sentiment_df = sentiment_df[sentiment_df['date'] >= datetime(2011, 1, 1).date()]
sentiment_df['date'] = sentiment_df['date'].astype(str)
df = pd.merge(df, sentiment_df, on='date', how='left').fillna(method='ffill')

In [ ]:
general_old_df = pd.read_csv('../../bitcoinforum_general/5_processing_extracted_data/general_old.csv')
general_old_df['date'] = pd.to_datetime(general_old_df['date']).dt.date
general_old_df = general_old_df[general_old_df['date'] >= datetime(2011, 1, 1).date()]
general_old_df.drop(columns=["posts_count","bitcoin_challenges","bitcoin_technology","bitcoin_adoption","educational_resources","altcoins"], inplace=True)
general_old_df['date'] = general_old_df['date'].astype(str)
general_old_df.tail(5)

In [ ]:
df = pd.merge(df, general_old_df, on='date', how='left').fillna(method='ffill')

In [ ]:
df.to_csv('monthly_stuff.csv', index=False)